# Tests of variable cell minimisation

In [ ]:
using JuLIP
using JuLIP.Potentials: StillingerWeber
using JuLIP.Solve: minimise!
using JuLIP.Constraints: VariableCell
using JuLIP.ASE: ASECalculator

using PyCall
using Plots
using Polynomials

@pyimport ase.units as units

# @pyimport quippy.potential as quippy_potential
# sw_pot = quippy_potential.Potential("IP SW")
# sw_calc_Q = ASECalculator(sw_pot)

sw_calc_J = StillingerWeber()

at = Atoms("Si")
#@assert energy(sw_calc_J, at) - energy(sw_calc_Q, at) < 1e-6

## Speed test

In [ ]:
Nsup = 10:20
Natoms = 2*Nsup.^3
t_J = []
t_Q = []
for N = Nsup
    push!(t_J, @elapsed forces(sw_calc_J, at * (N, N, N)))
    #push!(t_Q, @elapsed forces(sw_calc_Q, at * (N, N, N)))
end

plot(Natoms, [t_J], label=[:JuLIP :QUIP], marker=:o, 
    xlabel="N_atoms", ylabel="Time / s")

## Energy volume plot

In [ ]:
E = Float64[]
V = Float64[]
P = linspace(-0.01, 0.01, 5)
for p in P
    at = Atoms("Si")
    set_calculator!(at, sw_calc_J)
    set_constraint!(at, VariableCell(at, pressure=p))
    minimise!(at, verbose=2)
    V1 = det(defm(at))
    @printf("p=%.3f, V1=%.3f\n", p, V1)
    push!(E, energy(at))
    push!(V, V1)
end

## Quadratic fit

In [ ]:
p = polyfit(V, E, 2)
V0, = roots(polyder(p))
K = -p[1]
@printf("quadratic fit: V0=%.3f, Bulk modulus K = %.3f GPa", V0, K/units.GPa)

scatter(V, E, marker=:o)
plot!(v -> polyval(p, v), xlabel=:Volume, ylabel=:Energy, legend=false)

## Linear fit

In [ ]:
p = polyfit(V, collect(P), 1)
V0, = roots(p)
K = - V0 * polyval(polyder(p), V0)
@printf("linear fit: V0=%.3f A^3, Bulk modulus K = %.3f GPa", V0, K/units.GPa)

scatter(V, P)
plot!(x -> polyval(p, x), xlabel=:Volume, ylabel=:Pressure, legend=false)